# Corpus

This part of the work is based on two tutorial take from [here](https://radimrehurek.com/gensim/tut1.html) and [here](https://radimrehurek.com/gensim/tut2.html#available-transformations).



In [2]:
import pandas as pd                                     
import numpy as np                                      
import os 

import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim

from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities
from collections import defaultdict

### Clean data

We decide tu use only 'extractedBodyText' form the email beacause this flield is susceptible to contain more information about topics.

In [3]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
len(emails)

7945

We have 7945 emails but some of them doesn't have extractedBody, so we decide to drop them. Then we have 6742 emails left.

In [4]:
emails_text = emails['ExtractedBodyText'].dropna()
len(emails_text)

6742

For cleaning emails we first transfom all characters into lowercase and we transform all word u.s.a on usa and all u.s on us. After that we can remplace juste all punctuation signs by an espace. 

In [5]:
emails_text = emails_text.apply(lambda x: x.lower())
emails_text = emails_text.apply(lambda x: x\
                                .replace('u.s.a','usa')\
                                .replace('u.s','us'))
emails_text = emails_text.apply(lambda x: x\
                                .replace(':',' ')\
                                .replace('—',' ')\
                                .replace('-',' ')\
                                .replace('.',' ')\
                                .replace(',',' ')\
                                .replace('.',' ')\
                                .replace('<',' ')\
                                .replace('>',' ')\
                                .replace('=',' ')\
                                .replace('•',' ')\
                                .replace("\\",' ')\
                                .replace('\n', ' ')\
                                .replace('^',' ')\
                                .replace('\\',' ')\
                                .replace('?',' ')\
                                .replace('\'',' ')\
                               )
#emails_text = emails_text.apply(lambda x: x.replace('—',' '))


Because your emails_text is of type *pandas.core.series.Series* we want to convert it into list :

In [6]:
documents = emails_text.tolist()
type(documents)

list

We check that we still have 6742 emails

In [7]:
len(documents)

6742

The next step is to remove all common words. For that we create a basic stoplist from a STOPWORDS list. By iteration we have added some common words that appeared in the results. 

In [8]:
stoplist = set(STOPWORDS)
stoplist.add('pls')
stoplist.add('yes')
stoplist.add('call')
stoplist.add('pm')
stoplist.add('no')
stoplist.add('com')
stoplist.add('doc')
stoplist.add('docx')
stoplist.add('pdf')
stoplist.add('mr')
stoplist.add('mrs')
stoplist.add('call')
stoplist.add('need')
stoplist.add('one')
stoplist.add('two')
stoplist.add('fyi')
stoplist.add('00')
stoplist.add('will')
stoplist.add('know')
stoplist.add('re')
stoplist.add('ok')
stoplist.add('also')
stoplist.add('see')
stoplist.add('us')
stoplist.add('good')
stoplist.add('thx')
stoplist.add('new')
stoplist.add('go')
stoplist.add('you')
stoplist.add('now')
stoplist.add('done')
stoplist.add('yet')
stoplist.add('wrote')
stoplist.add('etc')
stoplist.add('back')
stoplist.add('today')
stoplist.add('said')
stoplist.add('many')
stoplist.add('already')
stoplist.add('want')


for i in range(10,100) :
    stoplist.add(i)



Now we uste the stoplist to create a list of word that are not in stoplist and that are bigger than 1 (this is inspired by the code of [tutorial of radimrehurek website](https://radimrehurek.com/gensim/tut1.html)

In [9]:
# remove list contains on stoplist and tokenize
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

For each word referenced we count how many time its appear (again code inspired of [tutorial of radimrehurek website](https://radimrehurek.com/gensim/tut1.html))

In [10]:
# counts the number of appartion of a word
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
# remove word that apprears only once
texts = [[token for token in text if frequency[token] > 1]
      for text in texts]        

We use now a dictionary to map the word with integer ids 

In [11]:
dictionary = corpora.Dictionary(texts)

In [12]:
print(dictionary)

Dictionary(17660 unique tokens: ['accident', 'shrewdly', '&c4', 'birch', 'remarks']...)


### Create Vector list

Now that we have a dictionary that contains uniques word, we can create your vector. The vector contains how many times each word appears in each documents (in your case one document is one email)

In [13]:
corpus = [dictionary.doc2bow(text) for text in texts]

### Apply transformation
We can now apply your Latent Dirichlet Allocation transformation. 

#### 5 topics

In [14]:
model_5 = models.LdaModel(corpus, id2word=dictionary, num_topics=5)


In [15]:
model_5.print_topics(5)

[(0,
  '0.007*"2010" + 0.005*"gov" + 0.005*"party" + 0.004*"tomorrow" + 0.004*"think" + 0.004*"4(d)" + 0.003*"time" + 0.003*"let" + 0.003*"b1" + 0.003*"cheryl"'),
 (1,
  '0.009*"2010" + 0.008*"gov" + 0.006*"b6" + 0.006*"hrod17@clintonemail" + 0.003*"december" + 0.003*"2009" + 0.003*"saturday" + 0.003*"state" + 0.003*"may" + 0.003*"10"'),
 (2,
  '0.004*"president" + 0.004*"obama" + 0.003*"people" + 0.003*"time" + 0.003*"women" + 0.003*"security" + 0.003*"policy" + 0.003*"state" + 0.003*"work" + 0.003*"american"'),
 (3,
  '0.026*"secretary" + 0.023*"office" + 0.019*"30" + 0.018*"state" + 0.014*"meeting" + 0.013*"10" + 0.013*"15" + 0.013*"room" + 0.012*"department" + 0.008*"45"'),
 (4,
  '0.006*"obama" + 0.004*"2010" + 0.004*"israel" + 0.004*"president" + 0.004*"american" + 0.003*"state" + 0.003*"government" + 0.003*"people" + 0.003*"republican" + 0.003*"print"')]

#### 20 topics

In [16]:
model_20 = models.LdaModel(corpus, id2word=dictionary, num_topics=20)

In [17]:
model_20.print_topics(20)

[(0,
  '0.009*"israel" + 0.007*"israeli" + 0.005*"talks" + 0.005*"afghan" + 0.004*"agree" + 0.004*"jewish" + 0.004*"russia" + 0.004*"karl" + 0.004*"settlements" + 0.003*"settlement"'),
 (1,
  '0.044*"2010" + 0.035*"gov" + 0.016*"hrod17@clintonemail" + 0.014*"fw" + 0.011*"cheryl" + 0.011*"b6" + 0.010*"sunday" + 0.010*"monday" + 0.009*"sullivan" + 0.009*"mills"'),
 (2,
  '0.010*"germany" + 0.008*"woodward" + 0.008*"ireland" + 0.007*"sure" + 0.007*"happy" + 0.006*"bibi" + 0.006*"time" + 0.005*"church" + 0.005*"book" + 0.004*"much"'),
 (3,
  '0.023*"state" + 0.014*"department" + 0.009*"house" + 0.007*"verveer" + 0.007*"secretary" + 0.006*"fco" + 0.006*"arsenal" + 0.005*"case" + 0.005*"gotten" + 0.004*"panel"'),
 (4,
  '0.015*"part" + 0.015*"b6" + 0.011*"release" + 0.008*"4(b)" + 0.007*"palestinians" + 0.007*"b5" + 0.006*"state" + 0.004*"settlements" + 0.004*"b1" + 0.004*"negotiating"'),
 (5,
  '0.023*"party" + 0.012*"labour" + 0.008*"david" + 0.007*"election" + 0.006*"vote" + 0.006*"bloomb

#### 35 topics

In [18]:
model_35 = models.LdaModel(corpus, id2word=dictionary, num_topics=35)

In [19]:
model_35.print_topics(35)

[(0,
  '0.019*"state" + 0.010*"funded" + 0.010*"changed" + 0.009*"department" + 0.009*"holiday" + 0.009*"getting" + 0.008*"panel" + 0.008*"date" + 0.006*"house" + 0.006*"(new"'),
 (1,
  '0.014*"kurdistan" + 0.012*"senate" + 0.009*"treaty" + 0.009*"office" + 0.008*"10" + 0.008*"state" + 0.007*"15" + 0.007*"white" + 0.007*"room" + 0.007*"12"'),
 (2,
  '0.014*"work" + 0.012*"give" + 0.012*"time" + 0.012*"make" + 0.011*"red" + 0.011*"talks" + 0.008*"cdm" + 0.008*"take" + 0.008*"next" + 0.008*"email"'),
 (3,
  '0.023*"office" + 0.013*"secretary" + 0.011*"30" + 0.011*"647" + 0.010*"dinner" + 0.009*"fm" + 0.009*"state" + 0.009*"house" + 0.009*"department" + 0.008*"nw"'),
 (4,
  '0.063*"b6" + 0.039*"part" + 0.035*"release" + 0.025*"b5" + 0.016*"discuss" + 0.014*"high" + 0.011*"importance" + 0.011*"mod" + 0.009*"message" + 0.009*"liked"'),
 (5,
  '0.006*"company" + 0.005*"last" + 0.005*"party" + 0.004*"even" + 0.004*"sid" + 0.004*"might" + 0.004*"time" + 0.004*"didn" + 0.004*"long" + 0.004*"wee

#### 50 topics

In [20]:
model_50 = models.LdaModel(corpus, id2word=dictionary, num_topics=50)

In [21]:
model_50.print_topics(50)

[(0,
  '0.015*"cameron" + 0.013*"week" + 0.011*"next" + 0.009*"mins" + 0.008*"tomorrow" + 0.008*"last" + 0.008*"passing" + 0.007*"move" + 0.007*"roger" + 0.007*"talk"'),
 (1,
  '0.030*"2010" + 0.019*"kurdistan" + 0.018*"settlements" + 0.016*"copy" + 0.014*"settlement" + 0.011*"ireland" + 0.010*"return" + 0.009*"10" + 0.009*"sudan" + 0.009*"soon"'),
 (2,
  '0.035*"clips" + 0.012*"2010" + 0.012*"industry" + 0.010*"sunday" + 0.010*"quartet" + 0.009*"stories" + 0.009*"arturo" + 0.009*"observations" + 0.008*"enterprise" + 0.008*"deal"'),
 (3,
  '0.013*"talks" + 0.013*"afghan" + 0.009*"un" + 0.008*"kabul" + 0.008*"tomorrow" + 0.007*"sunday" + 0.007*"2009" + 0.007*"december" + 0.007*"karzai" + 0.006*"bob"'),
 (4,
  '0.058*"israeli" + 0.032*"palestinian" + 0.013*"jewish" + 0.013*"funded" + 0.010*"film" + 0.009*"funds" + 0.008*"abu" + 0.008*"netanyahu" + 0.007*"occupation" + 0.007*"state"'),
 (5,
  '0.030*"note" + 0.020*"verveerms@state" + 0.017*"framework" + 0.014*"confirm" + 0.014*"melanne" +